In [1]:
# Libraries
import cobra
import pandas as pd
import numpy as np
import pyEQL.chemical_formula
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import sample
from  math import sqrt


#import csv
# function definitions

def GetComposition_old(formula,atom="C"):
    ch=""
    flag=False
    for c in formula:
        if c == atom:
            flag=True
        else:
            if c.isnumeric() and flag:
                ch=ch+c
            else:
                flag=False
    if ch == "":
        return 0
    else:
        return int(ch)

getSign=dict()

def getFBAvalues(model):
    #print("blalba")
    getSign.clear()
    fba = model.optimize()
    V=fba.fluxes
    for r in model.reactions:
        if V.get(r.id) > 0:
            getSign[r.id] = 1
        elif V.get(r.id) < 0:
            getSign[r.id] = -1
        else:
            getSign[r.id] = 0
    return model.optimize()
"""   
def GetComposition(m, atom="C"):
    #print("formula : ",formula)
    #if (formula == "X"):
        #print("////{}///" .format(formula))
        #return 3.26
    try:
        return pyEQL.chemical_formula.get_element_mole_ratio(m.formula,atom)
    except ValueError:
        #m.formula=m.formula.replace("R", "S") #provisoirement je remplace les atomes R et X qui ne st pas reconnus par S
        #m.formula=m.formula.replace("X", "S") #juste pour pouvoir faire de simulation
        #m.formula=m.formula.replace("L", "S") #juste pour pouvoir faire de simulation
        m.formula = m.formula.replace(m.formula, "S")
        if (m.formula == "S") and ("biomass" in m.id):
            for r in m.reactions:
                if "EX_" not in r.id:
                    toBiomass=0
                    for m2 in r.metabolites:
                        if "S" not in m2.formula:
                            toBiomass+= GetComposition(m2,atom)*r.get_coefficient(m2)
            form=atom+str(abs(round(toBiomass)))
            print("bbbbbbbbbbbbbbbbbbb{}" .format(toBiomass))
            #return abs(round(toBiomass))
            m.formula = m.formula.replace("S", form)
        else:
            form=atom+"0"
            m.formula = m.formula.replace("S", form)
        return pyEQL.chemical_formula.get_element_mole_ratio(m.formula,atom)
"""   
  
def GetComposition_new(m, atom):

    try:
        return pyEQL.chemical_formula.get_element_mole_ratio(m.formula,atom)
    except ValueError:
        m.formula = m.formula.replace(m.formula, "S5")
        return pyEQL.chemical_formula.get_element_mole_ratio(m.formula,atom)
    
def GetComposition(m, atom):  
    val=GetComposition_new(m, atom)
    if (m.formula == "S5"): 
        if ("biomass" in m.id):
            for r in m.reactions:
                if "EX_" not in r.id:
                    toBiomass=0
                    for m2 in r.metabolites:
                        if "S" not in m2.formula:
                            toBiomass+= GetComposition(m2,atom)*r.get_coefficient(m2)
            val=abs(toBiomass)
            print("bbbbbbbbbbbbbbbbbbb{}" .format(val))
        else:
            val=0
            print("cccccccccccc{}" .format(val))
    return val

    
    
def InverseMatrix(M):
    try:
        return np.linalg.inv(M) 
    except np.linalg.LinAlgError as e:
        if 'Singular matrix' in str(e):
            print('Singular matrix')
            return np.linalg.inv(np.eye(len(M[0, :]))) 

    
def getSign_j(r,fva=0):
    if fva == 0:
        minim=r.lower_bound
        maxim=r.upper_bound
    else:
        minim=fva["minimum"].get(r.id)
        maxim=fva["maximum"].get(r.id)
    if (abs(minim)+abs(maxim)>0.01):
        if (minim*maxim<0):
            return 0
        else:
            if (maxim<=0):
                return -1
            else:
                return 1
    else:
        return 0
            

ReactInputTable=dict()
ReactProductTable=dict()

def ReactionInputs(r, sign=1):
    if not ReactInputTable.get(r.id):
        ReactInputTable[r.id] = ReactionInputsLong(r,  sign)
    return ReactInputTable.get(r.id)
 
        
def ReactionProducts(r, sign=1):
    if not ReactProductTable.get(r.id):
        ReactProductTable[r.id] = ReactionProductsLong(r,sign)
    return ReactProductTable.get(r.id)
    
def ReactionInputsLong(r,sign=1):
    res=list()
    for m in r.metabolites:
        if (sign*r.get_coefficient(m)<-0.001):
            res.append(m)
    return res
        
def ReactionProductsLong(r, sign=1):
    res=list()
    for m in r.metabolites:
        if (sign*r.get_coefficient(m)>0.001):
            res.append(m)
    return res
    

def In(r,m,atom="C"):
    res=0
    if m in ReactionProducts(r,getSign[r.id]):
        res=abs(r.get_coefficient(m))*GetComposition(m, atom)
        somme=0
        for met in ReactionInputs(r,getSign[r.id]):
            somme+=abs(r.get_coefficient(met))*GetComposition(met,atom)
        if not somme == 0:
            res=res/somme
        else:
            res=1
    return res


def RescaleModel(model,frac=0.5):
    fva=cobra.flux_analysis.flux_variability_analysis(model,fraction_of_optimum=frac).to_dict()
    for r in model.reactions:
        r.lower_bound = fva["minimum"].get(r.id)
        r.upper_bound = fva["maximum"].get(r.id)
        print("{} {:.3f} {:.3f}" .format(r.id, r.lower_bound, r.upper_bound))
        
        
    
def value_of_D(mk,mi,V,atom="C"):
    # case 1 : r is the exchange reactions for m input
    if mk == mi:
        return 1
    # General case
    res=0
    for r in isConsumedBy[mi.id]:
            mr=metRate(mi,V)
            if not mr==0: 
                res -= abs(r.get_coefficient(mi)) * abs(V.get(r.id)) *In(r,mk,atom) / mr
    return res 



def getMetabolitLists(model, atom="C"):
    LMet=list()
    for m in model.metabolites:
        if (GetComposition(m, atom) != 0):
            if "biomass" in m.id:
                print("/////////biomassbiomass///////////////{}" .format(GetComposition(m, atom)))
            LMet.append(m)
    LMetInputs=list()
    for m in LMet:
        flag=False
        for r in m.reactions:
            if len(ReactionInputs(r,getSign[r.id]))==0 and m in ReactionProducts(r,getSign[r.id]):
                flag=True
        if (flag and GetComposition(m, atom) > 0.01):
            LMetInputs.append(m)       
    LMetOutputs=list()
    for m in LMet:
        flag=False
        for r in m.reactions:
            if len(ReactionProducts(r,getSign[r.id]))==0 and m in ReactionInputs(r,getSign[r.id]):
                flag=True
        if (flag and GetComposition(m, atom) > 0.01):
#            print(m.id,"is an output")
            LMetOutputs.append(m)

    LMetOthers=list()
    for m in LMet:
        # Beurkkkk !!!!
        flag=True
        for r in m.reactions:
            if len(ReactionProducts(r,getSign[r.id]))==0 and m in ReactionInputs(r,getSign[r.id]):
                flag=False
            if len(ReactionInputs(r,getSign[r.id]))==0 and m in ReactionProducts(r,getSign[r.id]):
                flag=False
        if (flag and GetComposition(m, atom) > 0.01):
#            print(m.id,"is an neither an input, nor an output")
            LMetOthers.append(m)
    LMet = LMetInputs+LMetOthers+LMetOutputs
    return LMet,LMetInputs,LMetOthers,LMetOutputs

def getMatD(model,V,atom):
    # get the list of metabolites
    LMet, LIn, LOthers, LOut = getMetabolitLists(model, atom)
    MatD=np.zeros((len(LMet),len(LMet)))
    i=0
    for mi in LMet:
        k=0
        for mk in LMet:
            MatD[k,i]=value_of_D(mk,mi,V,atom)
            k+=1
        i+=1
    return matD

def PrintRepartition(model,V,LMet, LIn, LOthers, LOut, atom="C",aff=True,precision=1e-10):
    ReactInputTable.clear()
    ReactProductTable.clear()
    EnrichModel(model)
    # get the list of metabolites
    #res=dict()
    #print("... Creating the metabolite lists")
    #LMet, LIn, LOthers, LOut = getMetabolitLists(model)
    print("... Creating matrix D")
    MatD=np.zeros((len(LMet),len(LMet)))
    i=0
    for mi in LMet:
        k=0
        for mk in LMet:
            MatD[k,i]=value_of_D(mk,mi,V,atom)
            k+=1
        i+=1
    print("... Inverting matrix D")
    invmatD=InverseMatrix(MatD)
    
    vect=list()
    D_LOut=len(LIn)+len(LOthers)
    #D_LOut=len(LIn)
    for i in range(D_LOut, len(LMet)):
        ch = "X_"+LMet[i].id+"*("+str(GetComposition(LMet[i],atom))+")"+" = "
        v=list()
        for k in range(len(LIn)):
            if (invmatD[i,k] > precision and (GetComposition(LMet[k],atom))>0.01):
                ch += " + "+str((round(invmatD[i,k],3)) )+" * X_"+LMet[k].id+"("+str(GetComposition(LMet[k],atom))+")"
                v.append( invmatD[i,k]*metRate(LIn[k],V)*GetComposition(LIn[k],atom) )
            else:
                v.append(0)
        vect.append(v)
        if aff:
            print(ch)
    ar = np.array(vect)
    #print(vect)
    AIO_matrice = pd.DataFrame(ar)
    return  AIO_matrice


def Estimation_MinMax_AIO(model, sampling_V, readLastMinMAx=False):
    if readLastMinMAx:
        Min_AIO_matrice = pd.read_csv('MinAIO_FirstSample.csv', sep =';', header = None)
        Max_AIO_matrice = pd.read_csv('MaxAIO_FirstSample.csv', sep =';', header = None)
        Min_AIO_matrice_Pred = Min_AIO_matrice.copy() 
        Max_AIO_matrice_Pred = Max_AIO_matrice.copy() 
    else:   
        V=sampling_V.loc[0]
        Min_AIO_matrice=PrintRepartition(model,V)
        Max_AIO_matrice=Min_AIO_matrice.copy()
    Nblig_AIO_matrice=len(Min_AIO_matrice[0])
    Nbcol_AIO_matrice=len(Min_AIO_matrice.loc[0])
    for id_V in range (1,len(sampling_V.index)):
        V=sampling_V.loc[id_V]
        AIO_matrice=PrintRepartition(model,V) 
        for l in range(Nblig_AIO_matrice):   #Pour la colonne fixée je parcours les elts
            for c in range(Nbcol_AIO_matrice):
                if (AIO_matrice[c][l] < Min_AIO_matrice[c][l]):
                    Min_AIO_matrice[c][l] = AIO_matrice[c][l]
                if (AIO_matrice[c][l] > Max_AIO_matrice[c][l]):
                    Max_AIO_matrice[c][l] = AIO_matrice[c][l]
    if readLastMinMAx:
        return Min_AIO_matrice_Pred, Max_AIO_matrice_Pred, Min_AIO_matrice, Max_AIO_matrice
    else:
        return Min_AIO_matrice, Max_AIO_matrice

def Sum_EuclidianDist_ColonneMatriceAIO(AIO_matrice):
    Nblig_AIO_matrice=len(AIO_matrice[0])
    Nbcol_AIO_matrice=len(AIO_matrice.loc[0])
    Sum_EuclidianDist_Colonne=0
    for c in range(Nbcol_AIO_matrice):
        EuclidianDist_Colonne=0
        for l in range(Nblig_AIO_matrice):   #Pour la colonne fixée je parcours les elts
            EuclidianDist_Colonne+= AIO_matrice[c][l]*AIO_matrice[c][l]
        EuclidianDist_Colonne= sqrt(EuclidianDist_Colonne)
        Sum_EuclidianDist_Colonne+=EuclidianDist_Colonne
    return  Sum_EuclidianDist_Colonne


isProducedBy=dict()
isConsumedBy=dict()

def EnrichModel(model):
    isProducedBy.clear()
    isConsumedBy.clear()
    for m in model.metabolites:
        isProducedBy[m.id]=list()
        isConsumedBy[m.id]=list()
        for r in m.reactions:
            if m in ReactionProducts(r,getSign[r.id]):
                isProducedBy[m.id].append(r)
            if m in ReactionInputs(r,getSign[r.id]):
                isConsumedBy[m.id].append(r)

def metRate(m,V):
    res=0
    for r in isProducedBy[m.id]:
        res+=abs(r.get_coefficient(m))*abs(V.get(r.id))
    return res

def metRate_inv(m,V):
    res=0
    for r in m.reactions:
        if m not in ReactionProducts(r,getSign[r.id]):
            res+=abs(r.get_coefficient(m))*V.get(r.id)
    return res

def FixModelFluxes(model,V):
    for r in model.reactions:
        value = V.get(r.id)
        r.lower_bound = value
        r.upper_bound = value
        
NutrFluxesInGutMicrobiote=dict() #ce dictionnaire est initialisé ds FixExFluxesGetExMetExAtom
                       # et utilisé ds les fnt FixLowerUpperBoundOfFixExchangeFluxe, FluxClassification
def FixExFluxesGetExMetExAtom(model,nutritionFile):
    NutrFluxesInGutMicrobiote.clear()
    NutrMetInGutMicrobiote=list()
    MetInGutMicrobioteAtom=list()
    fh = open(nutritionFile)
    reader = csv.reader(fh, delimiter = '\t')
    fluxesIdValues=list()
    fluxesId=list()
    fluxesValues=list()
    for ligne in reader:
        fluxesId.append(ligne[0][0:len(ligne[0])-3])
        fluxesValues.append(ligne[1])
    #Extraction de l'ID entre "EX_" et le 1er "[" ou "(" 
    for i in range(1, len(fluxesId)):
        InputId=fluxesId[i] + "_"
        #print(InputId)
        for r in model.reactions:
            if (InputId in r.id):
                NutrFluxesInGutMicrobiote[r]=float(fluxesValues[i])
    for r in NutrFluxesInGutMicrobiote:
    #print(r)
        for m in r.metabolites:
            NutrMetInGutMicrobiote.append(m)
            Latom=pyEQL.chemical_formula.get_elements(m.formula)
            for atom in Latom:
                if atom not in MetInGutMicrobioteAtom:
                    MetInGutMicrobioteAtom.append(atom)
            #print("{}; {}; {}" .format(r, m, pyEQL.chemical_formula.get_elements(m)  ))
    return NutrMetInGutMicrobiote, MetInGutMicrobioteAtom
    #return NutrFluxesInGutMicrobiote
#cette fonction prend en paramètre la valeur retournée par FixExchangeFluxes
def FixLowerUpperBoundOfFixExchangeFluxes(model):
    for r in NutrFluxesInGutMicrobiote: 
        React=model.reactions.get_by_id(r.id)
        React.lower_bound=-NutrFluxesInGutMicrobiote.get(r)
        #React.upper_bound=NutrFluxesInGutMicrobiote.get(r)
        #print("[{}: [{}  {}]" .format(r, React.lower_bound, React.upper_bound)) 

FluxImpIn_valMax=dict()
def FluxClassification(model, V, aff1=True):
    ImpIn=str("FLUX IMPORTES DANS LE REGIME")
    ImpOut=str("FLUX IMPORTES MAIS PAS DANS LE REGIME")
    ExpIn=str("FLUX EXPORTES  PRESENTS  DANS LE REGIME")
    ExpOut=str("FLUX EXPORTES  ET NON PRESENTS  DANS LE REGIME")
    Imp_valMax=str("flux importés à valeurs maximales")
    Imp_valNoMax=str("flux importés à valeurs NON nulles et NON maximales")
    Exp_valMax=str("flux exportés à valeurs maximales")
    Exp_valNoMax=str("flux exportés à valeurs NON nulles et NON maximales")
    In_NoImpExp=str(" PRESENTS  DANS LE REGIME MAIS NI IMPORTES, NI EXPORTES")
    Out_NoImpExp=str("NON PRESENTS  DANS LE REGIME ET NI IMPORTES, NI EXPORTES")

    #len(ReactionInputs(r,getSign[r.id]))==0  len(ReactionProducts(r,getSign[r.id])) == 0:
    FluxImpIn_valMax.clear()
    FluxImpIn_valNoMax=dict()
    FluxImpOut_valMax=dict()
    FluxImpOut_valNoMax=dict()
    FluxExpIn_valMax=dict()
    FluxExpIn_valNoMax=dict()
    FluxExpOut_valMax=dict()
    FluxExpOut_valNoMax=dict()
    

    FluxNulIn_NoImpExp=dict()
    FluxNulOut_NoImpExp=dict()
    total_Ex_reactNul=0
    total_reactNul=0
    precision=1e-6
    for r in NutrFluxesInGutMicrobiote:
        if abs(V.get(r.id)) <  precision:
            #FluxNulIn_NoImpExp[r]=list()
            FluxNulIn_NoImpExp[r.id]=V.get(r.id)
            total_Ex_reactNul+=1
        if (abs(NutrFluxesInGutMicrobiote.get(r)+V.get(r.id)) <  precision) and (abs(V.get(r.id)) >  precision):
            FluxImpIn_valMax[r.id]=list()
            FluxImpIn_valMax[r.id].append(V.get(r.id))
            FluxImpIn_valMax[r.id].append(NutrFluxesInGutMicrobiote.get(r))
        elif (abs(NutrFluxesInGutMicrobiote.get(r)+ V.get(r.id)) >=  precision ) and (V.get(r.id) < 0) :
            FluxImpIn_valNoMax[r.id]=list()
            FluxImpIn_valNoMax[r.id].append(V.get(r.id))
            FluxImpIn_valNoMax[r.id].append(NutrFluxesInGutMicrobiote.get(r))
    for r in model.reactions:
        React=model.reactions.get_by_id(r.id) #"EX_" in r.id 
        if "EX_" in r.id  and (abs(V.get(r.id)) <  precision) and (r not in NutrFluxesInGutMicrobiote): 
            #FluxNulOut_NoImpExp[r]=list()
            FluxNulOut_NoImpExp[r.id]=V.get(r.id)
            total_Ex_reactNul+=1
        if "EX_" in r.id  and (V.get(r.id) < 0) and (r not in NutrFluxesInGutMicrobiote):  
            if(abs(React.lower_bound - V.get(r.id)) <  precision): 
                FluxImpOut_valMax[r.id]=list()
                FluxImpOut_valMax[r.id].append(V.get(r.id))
                FluxImpOut_valMax[r.id].append(React.lower_bound)
            elif(abs(React.lower_bound - V.get(r.id)) >=  precision): 
                FluxImpOut_valNoMax[r.id]=list()
                FluxImpOut_valNoMax[r.id].append(V.get(r.id))
                FluxImpOut_valNoMax[r.id].append(React.lower_bound)
    for r in NutrFluxesInGutMicrobiote: 
        if (abs(NutrFluxesInGutMicrobiote.get(r)-V.get(r.id)) <  precision):
            FluxExpIn_valMax[r.id]=list()
            FluxExpIn_valMax[r.id].append(V.get(r.id))
            FluxExpIn_valMax[r.id].append(-NutrFluxesInGutMicrobiote.get(r))
        elif (abs(NutrFluxesInGutMicrobiote.get(r)- V.get(r.id)) >=  precision ) and (V.get(r.id) > 0) :
            FluxExpIn_valNoMax[r.id]=list()
            FluxExpIn_valNoMax[r.id].append(V.get(r.id))
            FluxExpIn_valNoMax[r.id].append(-NutrFluxesInGutMicrobiote.get(r))
    for r in model.reactions:
        React=model.reactions.get_by_id(r.id)
        if(abs(V.get(r.id)) <  precision): 
            total_reactNul+=1
        if "EX_" in r.id  and (V.get(r.id) > 0) and (r not in NutrFluxesInGutMicrobiote):  
            if(abs(React.upper_bound - V.get(r.id)) <  precision): 
                FluxExpOut_valMax[r.id]=list()
                FluxExpOut_valMax[r.id].append(V.get(r.id))
                FluxExpOut_valMax[r.id].append(React.upper_bound)
            elif(abs(React.lower_bound - V.get(r.id)) >=  precision): 
                FluxExpOut_valNoMax[r.id]=list()
                FluxExpOut_valNoMax[r.id].append(V.get(r.id))
                FluxExpOut_valNoMax[r.id].append(React.upper_bound)
    if aff1:
        print(ImpIn)
        print(Imp_valMax)
        if(len(FluxImpIn_valMax) > 0) : 
            print("NOMBRE EST: {}" .format(len(FluxImpIn_valMax)))
            for r in FluxImpIn_valMax:
                print("{} V={}  Vmax= {}" .format(r, FluxImpIn_valMax[r][0], FluxImpIn_valMax[r][1] ) )
        else:
            print("NEANT")
        print(Imp_valNoMax)
        if(len(FluxImpIn_valNoMax) > 0) :
            print("NOMBRE EST: {}" .format(len(FluxImpIn_valNoMax)))
            for r in FluxImpIn_valNoMax:
                print("{} V={}  Vmax= {}" .format(r, FluxImpIn_valNoMax[r][0], FluxImpIn_valNoMax[r][1] ) )
        else:
            print("NEANT")
        """
        Reste à afficher les flux ci-dessous
        FluxImpOut_valMax=dict()
        FluxImpOut_valNoMax=dict()
        FluxExpIn_valMax=dict()
        FluxExpIn_valNoMax=dict()
        FluxExpOut_valMax=dict()
        FluxExpOut_valNoMax=dict()
        """
        #print(pyEQL.chemical_formula.get_elements('FeSO4'))

        print("Total réactions = {}" .format(len(model.reactions)) )
        print("Total réactions Exchange nulles = {}" .format( total_Ex_reactNul ))
        print("Total réactions nulles = {}" .format( total_reactNul ))
        

def getNutrMaxFluxesAIO(model, V, nutritionFile, soucheName):

    NutrMetInGutMicrobiote, MetInGutMicrobioteAtom = FixExFluxesGetExMetExAtom(model,nutritionFile)
    del MetInGutMicrobioteAtom[MetInGutMicrobioteAtom.index('S')]
    print(MetInGutMicrobioteAtom)
    A=[ 'N', 'C']
    precision=1e-10
    start1 = time.time()
    #AIO_matrice=PrintRepartition(model,V, LMet,LMetInputs,LMetOthers,LMetOutputs,atom,aff,precision)


    fileName= soucheName+nutritionFile
    fileName=fileName.replace("Escherichia/", "")
    fileName=fileName.replace("Diet/", "")
    fileName=fileName.replace(".xml", "")
    fileName=fileName.replace(".csv", "")
    fileName=fileName+".csv"
    for atom in A:#MetInGutMicrobioteAtom:
        start2 = time.time()
        LMet, LIn, LOthers, LOut = getMetabolitLists(model, atom)

                    
        LFluxesInputs=list()
        LFluxesOutputs=list()
        isGutMicrobioteMaxFlux=list()
        LMetMaxFlux=list()
        LMaxFlux=list()
        new_LIn=list()
        if len(LOut) > 0:
            for m in LIn:
                for r in m.reactions:
                    #if "EX_" in r.id and  V.get(r.id) < 0 :
                    #if (len(ReactionInputs(r,getSign[r.id]))==0) :
                    if (len(ReactionInputs(r,getSign[r.id]))==0) and  m in ReactionProducts(r,getSign[r.id]) :
                        LFluxesInputs.append(abs(V.get(r.id)*r.get_coefficient(m)*GetComposition(m, atom)))
                        new_m = str(m) + ' [V=' + str(round(abs(V.get(r.id)*r.get_coefficient(m)),3))+',  '+ atom+ \
                        '=' + str(GetComposition(m, atom))+ ', flux=' + \
                        str(round( (GetComposition(m, atom)* abs(V.get(r.id)*r.get_coefficient(m))),3) )+']'
                        new_LIn.append(new_m)
                        #print("Input={}; Atome={}, flux={}" .format(m,atom, V.get(r.id)*r.get_coefficient(m)*GetComposition(m, atom) ))
                        if (r in NutrFluxesInGutMicrobiote) and abs(NutrFluxesInGutMicrobiote[r]+V.get(r.id)) < 1e-6:
                            isGutMicrobioteMaxFlux.append(1)

                        else:
                            isGutMicrobioteMaxFlux.append(0)

                for m in LOut: 
                    for r in m.reactions:
                        #if "EX_" in r.id and  V.get(r.id) < 0 :
                        #if (len(ReactionInputs(r,getSign[r.id]))==0) :
                        if len(ReactionProducts(r,getSign[r.id]))==0 and  m in ReactionInputs(r,getSign[r.id]) :
                            LFluxesOutputs.append(abs(V.get(r.id)*r.get_coefficient(m)*GetComposition(m, atom)))
                            #print("outp={}; Atome={}, flux={}" .format(m,atom, V.get(r.id)*r.get_coefficient(m)*GetComposition(m, atom) ))
                            #Inp = 27; FInp = 79; isGut= 134
            for i in range(len(new_LIn)):
                if isGutMicrobioteMaxFlux[i] == 1:
                    LMetMaxFlux.append(new_LIn[i])
                    LMaxFlux.append(LFluxesInputs[i])
            #print(LMetMaxFlux)
            #print(LMaxFlux)
            #print(new_LIn)

            AIO_matrice=PrintRepartition(model,V, LMet, LIn, LOthers, LOut ,atom)
            AIO_matrice.index=LOut
            AIO_matrice.columns=new_LIn
            #AIO_matrice.columns=LIn
            #print(AIO_matrice)
            
            AIOMaxFlux= abs(AIO_matrice.loc[:, LMetMaxFlux])
            AIOMaxFlux.to_csv(fileName, sep = ';', mode = 'a') 
            
            print(AIOMaxFlux)
            #AIOMaxFlux.to_csv("blo.csv", sep=";")
            for i in range(len(LMetMaxFlux)):
                AIOMaxFlux[LMetMaxFlux[i]] = AIOMaxFlux[LMetMaxFlux[i]].div(LMaxFlux[i])
            print("Affichage en pourcentage ....")
            print(AIOMaxFlux)
            AIOMaxFlux.to_csv(fileName, sep = ';', mode = 'a') 
            precision=1e-10
            end2 = time.time()
            print("     temps d'execution AIO ",end2 - start2," s")
    end1 = time.time()
    print("     temps d'execution AIO ",end1 - start1," s")
                                         
                

 

In [2]:
import time
import csv
import pyEQL.chemical_formula

LSoucheEcoli=["Escherichia/Escherichia_albertii_KF1.xml", "Escherichia/Escherichia_albertii_TW07627.xml", \
"Escherichia/Escherichia_coli_O157_H7_str_Sakai.xml", "Escherichia/Escherichia_coli_SE11.xml", \
"Escherichia/Escherichia_coli_str_K_12_substr_MG1655.xml", "Escherichia/Escherichia_coli_UTI89_UPEC.xml", \
"Escherichia/Escherichia_fergusonii_ATCC_35469.xml", "Escherichia/Escherichia_hermannii_NBRC_105704.xml", \
"Escherichia/Escherichia_sp_1_1_43.xml", "Escherichia/Escherichia_sp_3_2_53FAA.xml", \
"Escherichia/Escherichia_sp_4_1_40B.xml"]
"""
model=cobra.io.read_sbml_model("Escherichia/Escherichia_sp_4_1_40B.xml")
A=['C']
L=list()
for r in model.reactions:
    flag=False
    for m in r.metabolites:
        if ("X" in m.formula)  :
            flag=True
            print("{} ({})" .format(m, m.formula))
    if flag:
        print("{}" .format(r))
            
            
"""
            

A=["Escherichia/Escherichia_albertii_KF1.xml"]
for souche in A: #LSoucheEcoli:

    #model=cobra.io.read_sbml_model("Escherichia/Escherichia_albertii_KF1.xml")
    #model=cobra.io.read_sbml_model("Escherichia/Escherichia_albertii_TW07627.xml")
    #model=cobra.io.read_sbml_model("Escherichia/Escherichia_coli_O157_H7_str_Sakai.xml")
    #model=cobra.io.read_sbml_model("Escherichia/Escherichia_coli_SE11.xml")
    #model=cobra.io.read_sbml_model("Escherichia/Escherichia_coli_str_K_12_substr_MG1655.xml")
    #model=cobra.io.read_sbml_model("Escherichia/Escherichia_coli_UTI89_UPEC.xml") ##Singular matrix
    #model=cobra.io.read_sbml_model("Escherichia/Escherichia_fergusonii_ATCC_35469.xml") ##invalid element symbol
    #model=cobra.io.read_sbml_model("Escherichia/Escherichia_hermannii_NBRC_105704.xml")  ##invalid element symbol
    #model=cobra.io.read_sbml_model("Escherichia/Escherichia_sp_1_1_43.xml")
    #model=cobra.io.read_sbml_model("Escherichia/Escherichia_sp_3_2_53FAA.xml")
    model=cobra.io.read_sbml_model(souche)


    print("Nbre reactions: {}".format(len(model.reactions)))
    print("Nbre metabolites: {}".format(len(model.metabolites)))

    regime=['Diet/Vegetarian.tsv','Diet/EUaverage.tsv','Diet/Mediterranean.tsv','Diet/HighProtein.tsv', \
            'Diet/GlutenFree.tsv', 'Diet/HighFiber.tsv','Diet/Type2diabetes.tsv','Diet/HighFat_lowCarb.tsv',\
            'Diet/Unhealthy.tsv']
    B=['Diet/Vegetarian.tsv']
    #nutritionFile= 'fluxes.tsv'
    for nutritionFile in B: #regime:
        print("SOUCHE = {} ---> REGIME = {}" .format(souche, nutritionFile))
        NutrMetInGutMicrobiote, MetInGutMicrobioteAtom=FixExFluxesGetExMetExAtom(model,nutritionFile)
        FixLowerUpperBoundOfFixExchangeFluxes(model)

        start = time.time()
        fba=getFBAvalues(model)
        V=fba.fluxes
        end = time.time()
        #V.to_csv("bli.csv")
        print("       temps d'execution FBA: ",end - start," s")
        print("objectif value: {}".format(fba.objective_value))
        
        """ 
        for r  in model.reactions:
            tot=0
            for m in r.metabolites:
                tot+= GetComposition(m,"N")*r.get_coefficient(m)
            if abs(tot) > 1e-6:
                print("r={} TOT={} V={}" .format(r, tot, V.get(r.id)))
        """  
        """        
        tot=0
        for m in model.metabolites:
            for r in m.reactions:
                if "EX_"  in r.id :
                    tot+= GetComposition(m,"C")*r.get_coefficient(m)*V.get(r.id)
        print("//////{}///////" .format(tot)  )
        """
        
        getNutrMaxFluxesAIO(model, V, nutritionFile, souche)
        
        #FluxClassification(model,  V)
        """
        atom='C'
        for r in model.reactions:
        #if "biomass" in r.id:
            #print(r)
            total=0
            for m in r.metabolites:
                    total+= GetComposition(m.formula,atom)*r.get_coefficient(m)*V.get(r.id)
            if ("EX_" not in r.id) and (abs(total) > 1e-6):
                for m in r.metabolites:
                    print("m={} ({}={}), m.formula={}" .format(m, atom, GetComposition(m.formula,atom), m.formula)  )   
                print("r= {}, TOTAL = {}" .format(r, total)  )
        """
        """
        #print(model.metabolites.pnto_R__91__e__93__.summary())
        for m in model.metabolites:
            totalFlux=0
            for r in m.reactions:
                totalFlux+=r.get_coefficient(m)*V.get(r.id)*GetComposition(m.formula,"C")
                print(totalFlux)
            print("m={}, total={}" .format(m,totalFlux ))
        """
        
        
        """
        for r in model.reactions:
            if "biomass" in r.id:
                totalP=0
                totalM=0
                for m in r.metabolites:
                    #if "biomass" in m.id:
                    print("m={}, formule={}" .format(m, m.formula ))
                    val=r.get_coefficient(m)*V.get(r.id)*GetComposition(m.formula,"C")
                    if val > 0:
                        totalP+=val
                    else:
                        totalM+=val     
                        
                print("r={}, +={}, -={}, V={}" .format(r,totalP, totalM, V.get(r.id) ))
        
        """
        """
        for m in model.metabolites:
            if "X" in m.formula:
                for r in m.reactions:
                    #if V.get(r.id)>0:
                    print("m.name={}, m={}, formule={}  V={}" .format(m.name, m, m.formula, V.get(r.id) ))
        """
        """
        for m in model.metabolites:    
            if "R" in m.formula:
                for r in m.reactions:
                    if V.get(r.id)>0:
                        print("m.name={}, m={}, formule={}  V={}" .format(m.name, m, m.formula, V.get(r.id) ))
        """
        """
        for m in model.metabolites:
            if m.name=="Protein dithiol":
                #print("DNA replication")
                for r in m.reactions:
                    #if abs(V.get(r.id))>0:
                    print("m={}, r={}  V={}" .format(m, r, V.get(r.id) ))


        
        
        """
        """
        totalFluxOut=metRate(m,V,"C")
        totalFluxIn=metRate_inv(m,V, "C")

        #if abs(abs(totalFluxOut)-abs(totalFluxIn))>1e-6 :
        if abs(totalFluxOut) > 0 or abs(totalFluxIn) > 0  :
            if abs(abs(totalFluxOut)-abs(totalFluxIn)) > 1e-6 :
                print("des={}, m= {}({}), IN = {}, Out={}" .format(abs(abs(totalFluxOut)-abs(totalFluxIn)),m.id, GetComposition(m.formula,"C"), totalFluxIn, totalFluxOut))
                for r in m.reactions:
                    if abs(V.get(r.id)) > 1e-6:
                        print("r={},coef={}, V={}" .format(r, r.get_coefficient(m), V.get(r.id)))
        """
    

        #getNutrMaxFluxesAIO(model, V, nutritionFile)
        











Nbre reactions: 1724
Nbre metabolites: 1322
SOUCHE = Escherichia/Escherichia_albertii_KF1.xml ---> REGIME = Diet/Vegetarian.tsv
       temps d'execution FBA:  0.17184209823608398  s
objectif value: 12.06991776584612
['C', 'H', 'O', 'N', 'Ca', 'Cl', 'K', 'Mg', 'Na', 'P', 'Cu', 'Fe', 'Mn', 'Zn']
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
ccccc

cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb8.098603499999943
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb8.098603499999943
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb8.098603499999943
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
ccccccc

bbbbbbbbbbbbbbbbbbb8.098603499999943
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb8.098603499999943
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb8.098603499999943
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb8.

Affichage en pourcentage ....
                      arg_L__91__e__93__ [V=17.862,  N=4, flux=71.448]  \
biomass__91__c__93__                                          0.234077   
nac__91__e__93__                                              0.071956   
nh4__91__e__93__                                              0.162314   
orn__91__e__93__                                              0.341039   
ser_L__91__e__93__                                            0.111706   
uri__91__e__93__                                              0.065081   

                      pnto_R__91__e__93__ [V=0.018,  N=1, flux=0.018]  \
biomass__91__c__93__                                         0.353649   
nac__91__e__93__                                             0.117710   
nh4__91__e__93__                                             0.264346   
orn__91__e__93__                                             0.027938   
ser_L__91__e__93__                                           0.118813   
uri__91__e__9

bbbbbbbbbbbbbbbbbbb37.289708499999996
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb37.289708499999996
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb37.289708499999996
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbb

cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb37.289708499999996
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb37.289708499999996
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0


cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb37.289708499999996
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
bbbbbbbbbbbbbbbbbbb37.289708499999996
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
... Inverting matrix D
X_5drib__91__c__93__*(5) =  + 0.0 * X_4hbz__91__e__93__(7) + 0.0 * X_acgam__91__e__93__(8) + 0.0 * X_ala_D__91__e__93__(3) + 0.0 * X_alahis__91__e__93__(9) + 0.0 * X_arg_L__91__e__93__(6) + 0.0 * X_asn_L__91__e__93__(4) + 0.001 * X_cys_L__91__e__93__(3) + 0.0 * X_dad_2__91__e__93__(10) + 0.0 * X_dcyt__91__e__93__(9) + 0.0 * X_dgsn__91__e__

In [3]:
A2=['C', 'H', 'O', 'N', 'Ca', 'Cl', 'K', 'Mg', 'Na', 'P', 'Cu', 'Fe', 'Mn', 'Zn', 'S']
A=['C']
for atom in A:
    print("ATOME: {}" .format(atom))
    for r in model.reactions:
        if "biomass" in r.id:
            #print(r)
            total=0
            for m in r.metabolites:
                if "biomass" not in m.id :
                    total+= GetComposition(m,atom)*r.get_coefficient(m)*V.get(r.id)
            print("r= {}, TOTAL = {}" .format(r, total)  )
            
            
            
"""
A2=['C', 'H', 'O', 'N', 'Ca', 'Cl', 'K', 'Mg', 'Na', 'P', 'Cu', 'Fe', 'Mn', 'Zn']
A=['C']
for atom in A:
    
    totalFluxOut=0
    totalFluxIn=0
    print("ATOME: {}" .format(atom))
 
    for m in model.metabolites:
        for r in m.reactions:           
            if "EX_" in r.id and V.get(r.id)>0:
                #print('Output...')
                totalFluxOut+=abs(V.get(r.id)*r.get_coefficient(m)*GetComposition(m.formula,atom))
                #print("++{}: {}" .format(m, GetComposition(m.formula,atom)))
                #if GetComposition(m.formula,atom)== 0:
                    #print("////atom ={}; m= {}; formule= {}////" .format(atom, m, m.formula))
            if "EX_" in r.id and V.get(r.id)<0:
                #print('Input...')
                totalFluxIn+=abs(V.get(r.id)*r.get_coefficient(m)*GetComposition(m.formula,atom))
                #print("--{}: {}" .format(m, GetComposition(m.formula,atom)))
                #if GetComposition(m.formula,atom)== 0:
                    #print("////atom ={}; m= {}; formule= {}////" .format(atom, m, m.formula))
    if(abs(totalFluxIn-totalFluxOut)==0):
        print("atom={}, In={}, Out= {}" .format(atom, totalFluxIn, totalFluxOut))
    #else:
"""
        #print("fluxIn N EST PAS EGAL a fluxOut")
    
    #for m in model.metabolites:
        #totalFluxOut=metRate(m,V)
        #totalFluxIn=metRate_inv(m,V)
        #if "biomass" in r.id:
            #print("r={}, V={}" .format(r, V.get(r.id)))
"""
        for r in m.reactions:
            if r.get_coefficient(m) < 0:
                totalFluxIn+=abs(r.get_coefficient(m)*GetComposition(m.formula,atom))
            if r.get_coefficient(m) > 0:
                totalFluxOut+=abs(r.get_coefficient(m)*GetComposition(m.formula,atom))
"""


ATOME: C
r= EX_biomass__40__e__41__: biomass__91__c__93__ <=> , TOTAL = 0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
cccccccccccc0
r= biomass342: 0.0030965 10fthf__91__c__93__ + 0.0030965 2dmmq8__91__c__93__ + 0.0030965 5mthf__91__c__93__ + 0.0030965 ACP__91__c__93__ + 0.025011 PGP__91__c__93__ + 0.0030965 adocbl__91__c__93__ + 0.42793 ala_L__91__c__93__ + 0.0030965 amet__91__c__93__ + 0.2467 arg_L__91__c__93__ + 0.20083 asn_L__91__c__93__ + 0.20083 asp_L__91__c__93__ + 40.1102 atp__91__c__93__ + 0.0030965 ca2__91__c__93__ + 0.0030965 cl__91__c__93__ + 0.010648 clpn180__91__c__93__ + 0.010648 clpnai17__91__c__93__ + 0.010648 clpni17__91__c__93__ + 0.0030965 coa__91__c__93__ + 0.0030965 cobalt2__91__c__93__ + 0.025011 colipa__91__c__93__ + 0.084104 ctp__91__c__93__ + 0.0030965 cu2__91__c__93__ + 0.076146 cys_L__91__c__93__ + 0.016118 datp__91__c__93__ + 0.015923 dctp__91__c__93__ + 0.015923 dgtp__91__c__93__ + dnarep__91__c__93__ + 0.016118 dttp_

'\n        for r in m.reactions:\n            if r.get_coefficient(m) < 0:\n                totalFluxIn+=abs(r.get_coefficient(m)*GetComposition(m.formula,atom))\n            if r.get_coefficient(m) > 0:\n                totalFluxOut+=abs(r.get_coefficient(m)*GetComposition(m.formula,atom))\n'